# Notebook for Capstone Project

# Import libraries

In [66]:
import pandas as pd
import numpy as npù
!pip install bs4

from bs4 import BeautifulSoup
import requests

# Get the html from url

In [67]:
headers = {'Accept-Encoding': 'identity'}
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', headers=headers)

# Use BeatifulSoup library to analyze html and show it with prettify method

In [ ]:
soup = BeautifulSoup(r.text, 'html5lib')
print(soup.prettify())

# Parse the html in order to find infos as suggested and create the ngbr_toronto dataframe

In [69]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
ngbr_toronto=pd.DataFrame(table_contents)
ngbr_toronto['Borough']=ngbr_toronto['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

ngbr_toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


 # Group on PostalCode (it will automatically combine on the same Neighborhood values) and than replace all 'Not Assigned' values in Neighborhood with the value of the proper Borough 

In [70]:
ngbr_toronto = ngbr_toronto.groupby(by='PostalCode').sum()
ngbr_toronto['Neighborhood'] = ngbr_toronto.apply(lambda row: row['Borough'] if row['Neighborhood'] == 'Not Assigned' else row['Neighborhood'], axis=1)

# Print the shape of the created dataframe

In [71]:
ngbr_toronto.shape

(103, 2)